## Loading example dataset into database
1. Download MySQL dump file from [MySQL Public Tutorial](https://www.mysqltutorial.org/mysql-sample-database.aspx)
2. Create Cloud Storage bucket and Cloud SQL MySQL instance
3. Import MySQL dump file into Cloud SQL instance

### Database Schema
This is a very common example relational structure for a retail company
![alt text](images/MySQL-Sample-Database-Schema.png "Title")

### Set environment variables

In [ ]:
PROJECT_ID=config.project_id
REGION=config.region
INSTANCE_NAME=config.instance_name
DATABASE_VERSION=config.database_version
CPU = config.cpu
MEMORY = config.memory
STORAGE_SIZE = config.storage_size
DB_USER = config.db_user
DB_NAME = config.db_name
BUCKET = config.bucket

SQL_DATABASE_URL = 'https://www.mysqltutorial.org/wp-content/uploads/2018/03/mysqlsampledatabase.zip'
SQL_FILE = 'mysqlsampledatabase.sql'

SERVICE_ACCOUNT = config.service_account

# initialize parameters
INSTANCE_CONNECTION_NAME = f"{PROJECT_ID}:{REGION}:{INSTANCE_NAME}" # i.e demo-project:us-central1:demo-instance
print(f"Your instance connection name is: {INSTANCE_CONNECTION_NAME}")

In [ ]:
# Make the Cloud Storage bucket
!gsutil mb -l $REGION $BUCKET

Make the Cloud SQL instance
!gcloud sql instances create $INSTANCE_NAME \
  --database-version=$DATABASE_VERSION \
  --region=$REGION \
  --cpu=$CPU \
  --memory=$MEMORY \
  --storage-size=$STORAGE_SIZE

# Generic function to download the SQL dump file
def get_sql_data(sql_database_url):
    from io import BytesIO
    from urllib.request import urlopen
    from zipfile import ZipFile
    
    with urlopen(SQL_DATABASE_URL) as zipresp:
        with ZipFile(BytesIO(zipresp.read())) as zfile:
            zfile.extractall()

get_sql_data(SQL_DATABASE_URL)

# Copy the file to the Cloud Storage bucket
!gsutil cp $SQL_FILE $BUCKET

# Important: Allow the Cloud SQL service account to read from the bucket
!gsutil iam ch serviceAccount:$SERVICE_ACCOUNT:objectAdmin \
$BUCKET

# Import the SQL dump file into the Cloud SQL instance.
!gcloud sql import sql $INSTANCE_NAME $BUCKET/$SQL_FILE --quiet